In [10]:
import pandas as pd

In [11]:
remedy_file0 = pd.read_csv(r"C:\Users\A174411\Onedrive - Progressive Corporation\groupProject\Custom SQL Query (InformationTechnology)_Custom SQL Query.csv", dtype='object')

In [12]:
remedy_file = remedy_file0.copy(deep=True)
print(remedy_file.shape)
print(remedy_file.columns)

(225491, 12)
Index(['Assignee', 'Priority', 'Record Type', 'RecordID', 'Reported_Time',
       'Resolved_Date', 'Status', 'Status Reason', 'Submission Type',
       'Support_Group', 'Support_Org', 'Text'],
      dtype='object')


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\A174411\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [34]:
col = remedy_file["Text"]
col.head(10)

0                                                  NaN
1                                                  NaN
2                                                  NaN
3                                                  NaN
4                                                  NaN
5                                                  NaN
6           <SERVER> Reported no UF data for past hour
7           <SERVER> Reported no UF data for past hour
8    No KBA found for this. Received help from Ben ...
9    No KBA found for this. Received help from Ben ...
Name: Text, dtype: object

In [ ]:
token_list = []
for word in col:
    if word in token_list == "NaN":
        continue
    else:
        token_list.append(word)
    
token_list
    
#def convert(token_list):
   # return (str(token_list[0].split()))

#convert(token_list)
#tokens = nltk.sent_tokenize(token_list)
#tokens

In [7]:
import matplotlib.pyplot as plt
from nltk.probability import FreqDist

ModuleNotFoundError: No module named 'src'